# Open AI Gym

This Cpater is covering the OpenAI Gym API.
We implement randomly behaving Agent

## The anatomy of the agent
Defenitions:
* **Agent**: A person or a thing that takes an active role. In practice, it's some piece of code, 
    which implements some policy. Basically, this policy must decide what action is needed at every time step,
    given our observations.
* **Environment**: Some model of the world, which is external to the agent and has the responsibility of providing us
    with observations and giving us rewards. It changes its state based on our actions.
*  **Episodes**: the agent interactions with the environment is divided into a sequence of steps called episodes. Episodes can be finite, like in a game of chess, or infinite like the Voyager 2 mission.

Implemented in Python for a simplistic situation.
We will define an environment that gives the agent random rewards for a limited number of steps, regardless of the agent's actions. 
This scenario is not very useful, but will allow us to focus on specific methods in both the **environment** and the **agent** classes. 


### Environment Class

In [1]:
import random

class Environment:
    def __init__(self): #Intitialise the Environment method to initialize its internal state
        self.steps_left = 100 #In this case the state is just a counter that limiyts the number of steps the agent is allows to take to interact with the environment
    
    def get_observation(self): #Method to return the current environment's observation to the agent. It is usually implemented as some function of the internal state of the environment.  state:
        return [0.0, 0.0, 0.0] #In our example, the observation vector is always zero, as the environment basically has no interna state

    def get_actions(self): #Method to allow the agent to query the set of actions it can execute.
        #Normally, the set of actions that the agent can execute does not change over time, but some actions can become impossible in different states (for example, not every move is possible in any position of the TicTacToe game).
        return [0, 1] #In our case, there are only two actions that the agent can carry out, encoded with the integers 0 and 1:

    def is_done(self): #Method to indicate the end of the episode.
        return self.steps_left == 0 #TRue if steps_left eq 0

    def action(self, action): #Handles/Responds to the Agent's action and check if the episode is completed
        if self.is_done():
            raise Exception("Game is over")
        self.steps_left -= 1 #Else return a random reward number and decrement the steps_left - in this case the enviromnet ignors the Agent's action
        return random.random()      
        

### Agent Class

In [2]:
#In this example, the agent ingnores the observations received from the environment
#The agent selects actions randomly instead
class Agent:
    def __init__(self):
        self.total_reward = 0.0 #?Intialize total rewards

    def step(self, env): #Execute a step 
        current_obs = env.get_observation() #Requesrt observations from the environment
        actions = env.get_actions() ##Requesrt possible actions from the environment
        reward = env.action(random.choice(actions)) #Select random action and request the environment to execute it (through the action method)
        self.total_reward += reward #Increment total reward by the last step returned reward


### Main body of agent + environment

In [4]:
if __name__ == "__main__":
    env = Environment() # Create and instance of the Environment class 
    agent = Agent()     # Create and instance of the Agent class 

    while not env.is_done(): #Loop of the steps
        agent.step(env) #Agent executes the next step

    print("Total reward got: %.4f" % agent.total_reward)

Total reward got: 55.1000


## OpenAI Gym API
The Python library called Gym was developed and has been maintained by [OpenAI](www.openai.com). The main goal of Gym is to provide a rich collection of environments for RL experiments using a unified interface. So, it's not surprising that the central class in the library is an environment, which is called Env. It exposes several methods and fields that provide the required information about an environment's capabilities. From high level, every environment provides you with these pieces of information and functionality:
* A set of actions that are allowed to be executed in an environment. Gym supports both discrete and continuous actions, as well as their combination.
* The shape and boundaries of the observations that an environment provides the agent with.
* A method called step to execute an action, which returns the current observation, reward, and indication that the episode is over.
* A method called reset to return the environment to its initial state and to obtain the first observation.

Example:
Here we will import the Gym package and create an environment called CartPole. This environment is from the "classic control" group and its gist is to control the platform with a stick attached by its bottom part (see the following figure). The trickiness is that this stick tends to fall right or left and you need to balance it by moving the platform to the right or left on every step. The warning message we see is not our fault, but a small inconsistency inside Gym, which doesn't affect the result.
The observation of this environment is four float numbers containing information about the x coordinate of the stick's center of mass, its speed, its angle to the platform, and its angular speed. Of course, by applying some math and physics knowledge, it won't be complicated to convert these numbers into actions when we need to balance the stick, but our problem is much trickier: how do we learn to balance this system without knowing the exact meaning of the observed numbers and only by getting the reward? The reward in this environment is 1 given on every time step. The episode continues until the stick falls; so to get a more accumulated reward, we need to balance the platform in a way to avoid the stick falling.
This problem may look difficult, but in just two chapters we'll write the algorithm that will easily solve this CartPole in minutes, without any idea about what the observed numbers mean. We will do it only by trial-and-error and a bit of RL magic.
However, let's continue to play with our session:

In [5]:
import gym
env = gym.make('CartPole-v0') #Initaite the environment
print(env.action_space) #print the type of the action_scpace
print(env.observation_space) #print the type of the observation_space
obs = env.reset() #reset the env
obs # the observation is four numbers


Discrete(2)
Box(4,)


array([ 0.04123818, -0.00551832,  0.01500006, -0.02263831])

In [6]:
#Rerquest for a step in the environment
env.step(0) #request a step with an action - move left


(array([ 0.04112781, -0.20085214,  0.01454729,  0.27473931]), 1.0, False, {})

Here we pushed our platform to the left by executing the action 0 and got the tuple of four elements:
* A new observation that is a new vector of four numbers
* A reward of 1.0 
* The done flag = False, which means that the episode is not over yet and we're more or less okay
* Extra information about the environment that is an empty dictionary

We now use sample() method of the Space class on action_space and observation_space. This method returns a random sample from the underlying space, which in the case of our Discrete action space means a random number of 0 or 1 and for the observation space is a random vector of four numbers. The random sample of the observation space may not look useful, and this is true, but the sample from the action space could be used when we're not sure how to perform an action. This feature is especially handy for us, as we don't know any RL methods yet, but still want to play around with the Gym environment. Now we know enough to implement our first random-behaving agent for CartPole, so let's do it.

In [7]:
print(env.action_space.sample())
print(env.action_space.sample())
print(env.observation_space.sample()) 
print(env.observation_space.sample())

0
1
[  2.06581807e+00   6.99371253e+37   3.76012474e-02  -5.19578493e+37]
[  1.40058351e+00  -4.24759394e+37   3.28210980e-01   3.15552512e+38]


## The Random CartPole Agent
This is the agent code for the CartPole environment.
Here, we create the environment and initialize the counter of steps and the reward accumulator. On the last line, we reset the environment to obtain the first observation (which we'll not use, as our agent is stochastic):

In while loop, we sample a random action, then ask the environment to execute it and return to us the next observation(obs), the reward, and the done flag. If the episode is over, we stop the loop and show how many steps we've done and how much reward has been accumulated. 

In [8]:
import gym
if __name__ == "__main__": 
    env = gym.make("CartPole-v0") 
    total_reward = 0.0 
    total_steps = 0 
    obs = env.reset()
    
    while True:
        action = env.action_space.sample() #pick a random action from the environment
        obs, reward, done, _ = env.step(action) #exectute a step with the picked action
        total_reward += reward #Increment the the total reward by the newly received reward
        total_steps += 1  #increnment the total_steps
        if done:  #Check if the done flag hs been set to true - then break
            break 
    print("Episode done in %d steps, total reward %.2f" % (total_steps, total_reward))

Episode done in 24 steps, total reward 24.00


In [9]:
obs

array([ 0.14208131,  0.82689565, -0.22500035, -1.41514323])